In [1]:
import pandas as pd
import numpy as np

In [2]:
import random
from collections import deque
import tensorflow as tf
from tensorflow import keras

In [3]:
movie = pd.read_csv('movies.csv')
rating = pd.read_csv('ratings.csv')

In [4]:
movie['genres_split'] = movie['genres'].str.split('|')
genres_dummies = movie['genres_split'].explode().str.get_dummies().groupby(level=0).sum()
movie = movie[['movieId', 'title']].join(genres_dummies)

In [5]:
rating.drop(columns=['timestamp'], inplace=True)
movie = movie.drop(columns=['(no genres listed)'])

In [6]:
movie_1 = movie

In [7]:
movie_v1 = movie_1.drop(columns=['title'])

In [8]:
movie_v1

,movieId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62418,209157,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
62419,209159,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
62420,209163,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
62421,209169,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
data_all = movie_v1.merge(rating, on='movieId')

In [10]:
user1 = data_all[data_all['userId'] == 1]
user1.shape

(70, 22)

In [11]:
user1.nunique()

movieId        70
Action          2
Adventure       2
Animation       2
Children        2
Comedy          2
Crime           2
Documentary     2
Drama           2
Fantasy         2
Film-Noir       2
Horror          2
IMAX            1
Musical         2
Mystery         2
Romance         2
Sci-Fi          2
Thriller        2
War             2
Western         2
userId          1
rating          9
dtype: int64

### Env

In [12]:
class DQL_model:
    def __init__(self, state_size, action_size, epsilon= 0.6,gamma= 0.95 ,epsilon_min=0.01, epsilon_decay=0.995, learning_rate=0.001, batch_size=20):
        self.state_size = state_size
        self.action_size = action_size
        self.epsilon = epsilon
        self.gamma = gamma
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        
        self.memory = deque(maxlen=2000)
        
        self.model = self.build_model()
        
    def build_model(self):
        model = keras.Sequential()
        model.add(keras.layers.Dense(64, input_dim= self.state_size, activation= 'relu'))
        model.add(keras.layers.Dense(32, activation= 'relu'))
        model.add(keras.layers.Dense(self.action_size, activation= 'linear'))
        
        model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate= self.learning_rate))
        
        return model   
    
    def remember(self, state, action, reward, next_state):
        self.memory.append((state, action, reward, next_state))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.sample(range(self.action_size), 1)[0]
        state = state.astype(np.float32)
        act_values = self.model.predict(np.reshape(state, [1,self.state_size]))
        return np.argsort(act_values[0])[-1]
    
    def train(self):
        if len(self.memory) < self.batch_size:
            return
        minibatch = random.sample(self.memory, self.batch_size)
        
        for state, action, reward, next_state in minibatch:
            
            target = reward + self.gamma*np.amax(self.model.predict(np.reshape(next_state, [1,self.state_size]))[0])
            
            target_f = self.model.predict(np.reshape(next_state, [1,self.state_size])) 
            
            target_f[0][action] = target
            self.model.fit(np.reshape(state, [1,self.state_size]), target_f, epochs=1, verbose=0)
            
            print(target)           
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [53]:
class Env:
    def __init__(self,data_movie, data, user_id, dqn_model):
        self.data = data
        self.user_id = user_id
        self.data_movie = data_movie
        self.dqn_model = dqn_model
        
        self.user_data = self.data[self.data['userId'] == self.user_id]
        self.user_data.drop(columns=['userId'], inplace=True)
        
        self.genres = self.user_data.columns[1:-1]
        self.user_data.drop(columns=['movieId'], inplace=True)
        
        self.storage = self.user_data[self.genres]
        self.memory = None
        self.movie_suggestions = None
        
        self.skip_count = 0
        self.max_skip = 10
        
        self.get_memory_from_user_data()
        self.reset()
        
    def reset(self): 
        self.movie_suggestions = self.data_movie.sample(10)
        self.skip_count = 0
        return self.movie_suggestions

    
    def suggest_next_movie(self):
        if self.movie_suggestions is not None:
            self.reset()
        self.movie_suggestions = None
        for _ in range(5):
            current_state = self.memory[self.genres].values.astype(np.float32)
            
            suggested_genres_index = self.dqn_model.act(current_state)
            
            filtered_movie = self.data_movie[self.data_movie[self.genres[suggested_genres_index]] == 1]
        
            if not filtered_movie.empty:
                print(f"Genre: {suggested_genres_index}")
                self.movie_suggestions = pd.concat([self.movie_suggestions, filtered_movie.sample(1)])
            else:
                print("No movie found")
                self.movie_suggestions = pd.concat([self.movie_suggestions, self.data_movie.sample(1)])
        
        self.movie_suggestions = pd.concat([self.movie_suggestions, self.data_movie.sample(5)])
        
        return self.movie_suggestions
        
    
    def storage_data(self):
        if self.memory is not None:
            new_row = pd.DataFrame([self.memory[self.genres].values], columns=self.genres)
            self.storage = pd.concat([self.storage, new_row], ignore_index=True)
        return self.storage
    
    def reward_movie(self, rating):
        reward = rating
        
        if reward == 0:
            reward = 0.1
            
        elif reward < 3:
            reward = - (1 - reward/5)
        else:
            reward = reward/5
            
        return reward
    
    def rating_movie(self, rating= None):
        if rating is not None and self.memory is not None:
            
            self.memory = self.memory[self.genres]
            
            state = self.memory.values.astype(np.float32)
            
            action = self.dqn_model.act(state)
            
            reward = self.reward_movie(rating)
            
            next_state = state
            
            self.dqn_model.remember(state, action, reward, next_state)
            
            self.dqn_model.train()
            
            return self.memory
    
    def step(self, index = None):
        if index is not None and self.movie_suggestions is not None:
            self.selected_movie = self.movie_suggestions.iloc[index]
            
            self.memory = self.selected_movie
            return self.selected_movie
        else:
            self.skip_count += 1
            if self.skip_count == self.max_skip:
                self.reset()
            
        
    def get_memory_from_user_data(self):
        for _, row in self.user_data.iterrows():
            state = row[self.genres].values
            
            action = [index for index, value in enumerate(state) if value == 1]
            
            if action:
                action = random.choice(action)
            
            rating = row.max()
            
            reward = self.reward_movie(rating)
            
            random_index = np.random.randint(0, len(self.user_data))
            next_state = self.user_data.iloc[random_index][self.genres].values
            
            self.dqn_model.remember(state, action, reward, next_state)
            
        self.dqn_model.train()
    
    

In [54]:
state_size=19
action_size=19
dql = DQL_model(state_size, action_size)
env = Env(movie,data_all, user_id= 1, dqn_model= dql)

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\DELL\AppData\Local\Temp\ipykernel_22964\1218094007.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.user_data.drop(columns=['userId'], inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_22964\1218094007.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1.3443952262401582
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
0.8646600686013698
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1.1062507979571818
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.9076132874935866
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
-0.3923094626516104
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1.0070030681788922
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1.1603418350219727
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
-0.3881504490971565
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.7133594054728746
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.8150693848729134
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━

In [55]:
env.step(1)

movieId                     146016
title          Elder Sister (1966)
Action                           0
Adventure                        0
Animation                        0
Children                         0
Comedy                           0
Crime                            0
Documentary                      0
Drama                            1
Fantasy                          0
Film-Noir                        0
Horror                           0
IMAX                             0
Musical                          0
Mystery                          0
Romance                          1
Sci-Fi                           0
Thriller                         0
War                              0
Western                          0
Name: 34672, dtype: object

In [56]:
title = movie[movie['movieId'] == env.selected_movie['movieId']]
title

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
34672,146016,Elder Sister (1966),0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [57]:
env.rating_movie(rating=2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1.011590338498354
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1.069672518968582
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1.0881752245128156
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
0.8961128063499927
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
0.9086893849074841
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
-0.34091798812150953
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
-0.6913431882858276
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1.1847047992050648
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
0.9348105818033219
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1.1922420889139176
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━

Action         0
Adventure      0
Animation      0
Children       0
Comedy         0
Crime          0
Documentary    0
Drama          1
Fantasy        0
Film-Noir      0
Horror         0
IMAX           0
Musical        0
Mystery        0
Romance        1
Sci-Fi         0
Thriller       0
War            0
Western        0
Name: 34672, dtype: object

In [58]:
env.memory.values

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
      dtype=object)

In [59]:
env.storage_data()
env.storage

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0
67,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0
68,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
69,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [60]:
env.suggest_next_movie()

Genre: 15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Genre: 2
Genre: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Genre: 2
Genre: 4


,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
21446,110508,Trancers II (1991),1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6413,6536,Sinbad: Legend of the Seven Seas (2003),0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53077,186677,Thenmavin Kombath (1994),0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41770,162416,Big Fish & Begonia (2016),0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45101,169624,Kurt Metzger: White Precious (2014),0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43337,165793,Out West with the Hardys (1938),0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37097,151903,Amazons (1986),1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
10252,37739,"Greatest Game Ever Played, The (2005)",0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
21396,110281,King of Comedy (Hei kek ji wong) (1999),0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
44618,168618,As You Are (2016),0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


### test2

In [61]:
env.movie_suggestions[env.genres]

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
21446,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
6413,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
53077,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41770,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
45101,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
43337,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
37097,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
10252,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
21396,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
44618,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [62]:
env.step(1)

movieId                                           6536
title          Sinbad: Legend of the Seven Seas (2003)
Action                                               0
Adventure                                            1
Animation                                            1
Children                                             1
Comedy                                               0
Crime                                                0
Documentary                                          0
Drama                                                0
Fantasy                                              1
Film-Noir                                            0
Horror                                               0
IMAX                                                 0
Musical                                              0
Mystery                                              0
Romance                                              0
Sci-Fi                                               0
Thriller  

In [63]:
title = movie[movie['movieId'] == env.selected_movie['movieId']]
title

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
6413,6536,Sinbad: Legend of the Seven Seas (2003),0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
env.rating_movie(rating=4)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
-0.6842815507203341
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
0.9337115295231342
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
-0.3820944752544165
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1.1187515005469322
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.8874979406595229
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
-0.08554811626672748
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
0.8213003352284431
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1.1980447888374328
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
0.7660242356359959
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1.158042798936367
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━

Action         0
Adventure      1
Animation      1
Children       1
Comedy         0
Crime          0
Documentary    0
Drama          0
Fantasy        1
Film-Noir      0
Horror         0
IMAX           0
Musical        0
Mystery        0
Romance        0
Sci-Fi         0
Thriller       0
War            0
Western        0
Name: 6413, dtype: object

In [65]:
env.storage_data()
env.storage

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0
68,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
69,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
70,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [66]:
env.suggest_next_movie()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Genre: 2
Genre: 7
Genre: 16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Genre: 2
Genre: 4


,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
18309,95856,Knick Knack (1989),0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12820,63840,Kanak Attack (2000),0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
51844,184017,Mute (2018),0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,0,0
45042,169506,Asteria (2017),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2839,2931,Time of the Gypsies (Dom za vesanje) (1989),0,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
11322,50517,Time (Shi gan) (2006),0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
47543,174773,Kino-pravda no. 15 (1925),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38528,155016,The Bus Is Coming (1971),1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
28396,131612,An American Summer (1991),0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
49224,178411,Thunder Cops II (1989),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
